In [1]:
import os
import time

import numpy as np
import ml_util_assignment_2 as utl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, validation_curve
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.metrics import log_loss, f1_score, accuracy_score

import mlrose_hiive
from mlrose_hiive import GeomDecay, ArithDecay, ExpDecay
from mlrose_hiive.runners import NNGSRunner
from mlrose_hiive.neural.activation import relu
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

save_directory = "figures/NeuralNetwork"
model_name = "NeuralNetwork"

folders = ["figures/NeuralNetwork/Complexity_Analysis",
           "figures/NeuralNetwork/Grid_Search_Results",
           "figures/NeuralNetwork/Learning_Curves",
           "figures/NeuralNetwork/Confusion_Matrix",
           "figures/NeuralNetwork/Metrics"]

directories = {
    "Save Directory": "figures/NeuralNetwork",
    "Initial Complexity Analysis": "figures/NeuralNetwork/Initial Complexity Analysis",
    "Grid Search Results": "figures/NeuralNetwork/Grid Search Results",
    "Learning Curves": "figures/NeuralNetwork/Learning Curves",
    "Final Complexity Analysis": "figures/NeuralNetwork/Final Complexity Analysis"
}

Random_Number = 42
TESTING = True
cv = 5
n_jobs = -1
np.random.seed(42)
get_ipython().system('pip install pyarrow')

<Figure size 1600x1200 with 0 Axes>

In [2]:
gathered_data = utl.setup(["MNIST"])
# gathered_data_fashion = utl.setup(["Fashion-MNIST"])
train_X, train_y, valid_X, valid_y, test_X, test_y = utl.split_data(gathered_data["MNIST"]["X"],
                                                                    gathered_data["MNIST"]["y"], minMax=True, oneHot=True)
# fashion_train_X, fashion_train_y, fashion_valid_X, fashion_valid_y, fashion_test_X, fashion_test_y = utl.split_data(
#     gathered_data_fashion["Fashion-MNIST"]["X"], 
#     gathered_data_fashion["Fashion-MNIST"]["y"], minMax=True)

dataset folder already exists.
MNIST dataset found:
	Loading MNIST.feather
	Finished loading MNIST dataset


In [3]:
CHECK_FOLDER = os.path.isdir(save_directory)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(save_directory)
    print("created folder : ", save_directory)
else:
    print(save_directory, "folder already exists.")

for f in folders:
    if not os.path.isdir(f):
        os.makedirs(f)
        print("created folder : ", f)
    else:
        print(f, "folder already exists.")

figures/NeuralNetwork folder already exists.
figures/NeuralNetwork/Complexity_Analysis folder already exists.
figures/NeuralNetwork/Grid_Search_Results folder already exists.
figures/NeuralNetwork/Learning_Curves folder already exists.
figures/NeuralNetwork/Confusion_Matrix folder already exists.
figures/NeuralNetwork/Metrics folder already exists.


# Supervised Learning Neural Network Parameters

In [4]:
hidden_layer_sizes=[[100]] # was (100,) in previous but mlrose needs a list

random_state=42

val = 3000
pred_val = 3000

activation='relu'

max_iterations = 5000

In [ ]:
type(train_y)

In [ ]:
train_X.iloc[:val, :].shape

In [13]:

grid_search_parameters = ({
    'max_iters': [1000],
    'learning_rate': 10. ** np.arange(-4,4,1) # nn params
})

nn_rhc_gs = NNGSRunner(x_train=train_X.iloc[:val, :].to_numpy(),
                       y_train=train_y.iloc[:val, :].to_numpy(),
                       x_test=test_X.iloc[:val, :].to_numpy(),
                       y_test=test_y.iloc[:val, :].to_numpy(),
                       experiment_name='nn_rhc_test',
                       activation=[relu],
                       algorithm=mlrose_hiive.algorithms.rhc.random_hill_climb,
                       grid_search_scorer_method=accuracy_score,
                       grid_search_parameters=grid_search_parameters,
                       iteration_list=[10, 50, 100, 300, 400, 500, 600, 700, 800, 900, 1000],
                       hidden_layer_sizes=hidden_layer_sizes,
                       bias=True,
                       early_stopping=True,
                       clip_max=1e+10,
                       max_attempts=500,
                       generate_curves=True,
                       output_directory=f"{os.getcwd()}/results/",
                       n_jobs=6,
                       cv=5,
                       seed=3)
rhc_res = nn_rhc_gs.run()

Running nngs_rhc
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed: 18.9min finished


Run time: 1135.9817591
Saving: [C:\Users\joshu\OneDrive - Georgia Institute of Technology\Georgia-Tech\CS 7641 - Machine Learning\Assignments\Randomized Optimization/results/nn_rhc_test\nngs_rhc__nn_rhc_test__cv_results_df.csv]
Saving: [C:\Users\joshu\OneDrive - Georgia Institute of Technology\Georgia-Tech\CS 7641 - Machine Learning\Assignments\Randomized Optimization/results/nn_rhc_test\nngs_rhc__nn_rhc_test__grid_search_results.p]
**********
Score: nan
**********
********************************
*** Spawn Count Remaining: 2 ***
********************************


In [14]:
rhc_res

(Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
 0     322.220272      0.379883         0.009801        0.000981   
 1     312.530259      5.337827         0.008200        0.001327   
 2     309.058902      7.642343         0.010602        0.002579   
 3     262.598546     49.077920         0.006997        0.000632   
 
   param_activation param_hidden_layer_sizes  param_learning_rate  \
 0             relu                    [100]               0.1000   
 1             relu                    [100]               0.0100   
 2             relu                    [100]               0.0010   
 3             relu                    [100]               0.0001   
 
    param_max_iters                                             params  \
 0             1000  {'activation': <function relu at 0x000001D779C...   
 1             1000  {'activation': <function relu at 0x000001D779C...   
 2   

# Randomized Hill Climb Optimization

In [ ]:
nn_model_random_hill = mlr_h.NeuralNetwork(hidden_nodes = hidden_layer_sizes, activation = activation, 
                                            algorithm = 'random_hill_climb', max_iters = max_iterations, 
                                            bias = True, is_classifier = True, learning_rate = 10.0,
                                            early_stopping = True, clip_max = 5, max_attempts = 1000, 
                                            random_state = random_state, curve=True)

In [ ]:
nn_model_random_hill.fit(fashion_train_X.iloc[:val, :], fashion_train_y.iloc[:val])

In [ ]:
y_pred = nn_model_random_hill.predict(fashion_train_X.iloc[:val, :])
y_train_acc = accuracy_score(np.argmax(fashion_train_y.iloc[:val,:].to_numpy(), axis=1), np.argmax(y_pred, axis=1))
print(y_train_acc)

In [ ]:
np.argmax(y_pred, axis=1)

In [ ]:
np.argmax(fashion_train_y.iloc[:val,:].to_numpy(), axis=1)

In [ ]:
print(nn_model_random_hill.node_list)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(0,max_iterations),nn_model_random_hill.fitness_curve)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(0,max_iterations),nn_model_random_hill.fitness_curve)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(0,max_iterations),nn_model_random_hill.fitness_curve)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(0,1000),nn_model_random_hill.fitness_curve)

# Simulated Annealing Optimization

# Genetic Algortihm Optimization